In [1]:
import pandas as pd
import plotly.express as px
import json
import plotly.graph_objects as go

In [2]:
fragment_file = r'data/instances_regular/fragments/f-I-5-5-200-10.json'
fragment_file = r'data/instances_large/fragments/f-I-7-7-1000-01.json'
fragment_data = json.load(open(fragment_file))
# instance_data = json.load(open('data/instances_regular/I-1-1-50-01.json'))
print(len(fragment_data['fragments']), len(fragment_data['contracted_fragments']),  len(fragment_data['contracted_fragments'])/len(fragment_data['fragments']) *100)

1725985 35903 2.0801455400829094


In [4]:
fragments = fragment_data["fragments"]
# Converting each fragment into its jobs
job_fragment = []
for i, fragment in enumerate(fragments):
    if i > 20:
        break
    jobs = []
    for j, job in enumerate(fragment["jobs"]):
        job_fragment.append({"fragment": str(i), "id": job})

job_df = pd.DataFrame(job_fragment)

In [ ]:
job_df

In [ ]:
job_data = instance_data["jobs"]
job_by_id = {job["id"]: job for job in job_data}

In [ ]:
job_df["start_time"] = job_df["id"].map(lambda x: job_by_id[x]["start_time"])
job_df["end_time"] = job_df["id"].map(lambda x: job_by_id[x]["end_time"])

In [ ]:
px.scatter(job_df, x="start_time", y="end_time", color="fragment", title="Fragment Start Times", labels={"start_time": "Start Time", "fragment": "Fragment"}).show()

In [ ]:
fig = go.Figure()

for i, fragment in enumerate(fragments):
    if i > 100:
        break
    frag_x = []
    frag_y = []
    name = str(i)
    for job in fragment["jobs"]:
        job_data = job_by_id[job]
        frag_x.append(job_data["start_time"])
        frag_y.append(job_data["end_time"])
    fig.add_trace(go.Scatter(x=frag_x, y=frag_y, mode="lines+markers", name=name))

increment = []
print(len(fig.data), len(fragments))
for i in range(0, len(fig.data)):
    step = dict(method="update",
                    args=[{"visible": [False] * len(fig.data)},
                          {"title": i}],
                    label=i
                    )
    
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    # step["args"][0]["visible"][0] = True  # Toggle i'th trace to "visible"
    increment.append(step)
sliders = [dict(active=0,
                currentvalue={"prefix": "Fragment ID: "},
                pad={"t": 50},
                steps=increment)]
fig.update_layout(
        sliders=sliders
    )
for i in fig.data:
    i.visible = False
fig.data[0].visible = True
# fig.data[1].visible = True
fig.show()


In [ ]:
from math import comb
sum(5*4*round(comb(50, i)) for i in range(5))

### Breakdown of fragment similarity

In [ ]:
#Number of jobs in each fragment
jobs_in_fragment = [len(fragment["jobs"]) for fragment in fragments]
px.histogram(jobs_in_fragment, title="Number of Jobs in Each Fragment", labels={"value": "Number of Jobs", "count": "Number of Fragments"}).show()


#Number of fragments with the same jobs

### Number of identical jobs between fragments

In [ ]:
from collections import defaultdict

fragment_job_duplicates = defaultdict(int)

for fragment in fragments:
    fragment_job_duplicates[tuple(fragment["jobs"])] += 1
for f in fragment_job_duplicates:
    fragment_job_duplicates[f] -= 1
print(sum(fragment_job_duplicates.values())/ len(fragments))
# px.histogram(list(fragment_job_duplicates.values()), title="Number of Fragments with the Same Jobs", labels={"value": "Number of Fragments", "count": "Number of Jobs"}).show()
print(min(fragment_job_duplicates.values()), max(fragment_job_duplicates.values()))


In [5]:
print(len(fragment_data["contracted_fragments"]))

297381


In [6]:
print(len(fragment_data["fragments"]))

5730816
